Notebook example that shows how datashader can be used to produce a plot (from the data obtained in previous python notebook: Bike Vis.ipynb)

In [27]:
from bokeh.models import GeoJSONDataSource
from bokeh.io import output_notebook, output_file, save
from bokeh.sampledata.sample_geojson import geojson
from datashader.bokeh_ext import InteractiveImage
from datashader.utils import export_image
from functools import partial
import bokeh.plotting as bp
import datashader.transfer_functions as tf
import datashader as ds
import pandas as pd
import numpy as np
#import ogr2ogr
import json

In [2]:
output_notebook()

Loading BokehJS ...

### Test out a poly-plot to see how it works

In [3]:
geo_source = GeoJSONDataSource(geojson=geojson)

In [4]:
for it in json.loads(geojson):
    print(it)

features
type


In [5]:
json.loads(geojson)['type']

'FeatureCollection'

In [6]:
len(json.loads(geojson)['features'])

25

In [7]:
json.loads(geojson)['features'][0]

{'geometry': {'coordinates': [-2.1208465099334717, 51.4613151550293],
  'type': 'Point'},
 'id': '463098',
 'properties': {'Address1': '1st Floor',
  'Address2': 'Bewley House',
  'Address3': 'Marshfield Road',
  'City': 'Chippenham',
  'County': 'Wiltshire',
  'Email': 'england.contactus@nhs.net',
  'IsPimsManaged': 'True',
  'OrganisationCode': 'Q64',
  'OrganisationName': 'Bath, Gloucestershire, Swindon And Wiltshire Area Team',
  'OrganisationStatus': 'Visible',
  'OrganisationType': 'Area Team',
  'Phone': '0113 8251 500',
  'Postcode': 'SN15 1JW',
  'SubType': 'UNKNOWN',
  'Website': 'http://www.england.nhs.uk/south/south/bgsw-at/'},
 'type': 'Feature'}

In [8]:
plot_json = json.loads(geojson)
plot_json['features'] = plot_json['features'][:5]  # Just 5

In [9]:
# Kick back to json
geo_source = GeoJSONDataSource(geojson=json.dumps(plot_json))

In [10]:
# Cool we have just 5
p = bp.figure(plot_width=300, plot_height=200)
p.circle(x='x', y='y', alpha=0.9, source=geo_source)
bp.show(p)

### Main Bike Data

In [34]:
plot_h = 625
plot_w = 1000

In [35]:
routes = pd.read_csv('full_16m_routes.csv')

df = routes.loc[:,('lat', 'lon', 'c', 'flag')]
# Square count to get higher diff
df['c'] = np.square(df.c)

print(df.shape)
df.head()

(8686359, 4)


,lat,lon,c,flag
0,51.490397,-0.122587,3600,NaN
1,51.491020,-0.122305,3600,NaN
2,51.491462,-0.121699,3600,NaN
3,51.492347,-0.121343,3600,NaN
4,51.492266,-0.120971,3600,NaN


In [36]:
df.c.min(), df.c.max()

(1, 82664464)

In [37]:
x_range = (df.lon.min(), df.lon.max())
y_range = (df.lat.min(), df.lat.max())

print(x_range)
print(y_range)

(-0.23794699999999999, 0.0079649999999999999)
(51.449814000000003, 51.553406000000003)


In [38]:
# We need to clip the shapefile:
#-clipdst [xmin ymin xmax ymax]
#ogr2ogr.main(["", "-f", "GeoJSON", "-clipdst",
#              "-0.237947", "51.449814", "0.007965", "51.553407",
#              "Basemaps/buildings_city.geojson", 
#              "Basemaps/TQTL_LON.shp", "-t_srs", "EPSG:4326"])

# Try using matplotlib later:
# https://anaconda.org/jbednar/census-hv-dask/notebook
# Then no need to convert

In [39]:
# Load buildings but truncate heavily to 100 polygons (at first)
with open('Basemaps/buildings_city.geojson', 'r') as f:
    geojson_buildings = f.read()

In [40]:
geo_buildings = json.loads(geojson_buildings)
# Clipped buildings is smaller and can be plotted
print(len(geo_buildings['features']))  #15,391
#geo_buildings['features'] = geo_buildings['features'][:100]  # clip for test

15391


In [41]:
# Check out what it looks like
geo_buildings['features'][0]

{'geometry': {'coordinates': [[[-0.178741503922634, 51.45247303835862],
    [-0.178892696351749, 51.452368770711665],
    [-0.179076684540893, 51.45218062216903],
    [-0.178810173993229, 51.45207888407964],
    [-0.178960235245966, 51.451923876661006],
    [-0.179198803720819, 51.4520147683327],
    [-0.17939614283654, 51.45181464729954],
    [-0.179652394271227, 51.45190829413508],
    [-0.181565113005794, 51.449814],
    [-0.181897849628305, 51.449814],
    [-0.179113833280065, 51.4526837945696],
    [-0.178741503922634, 51.45247303835862]]],
  'type': 'Polygon'},
 'properties': {'ID': 91530},
 'type': 'Feature'}

In [42]:
# Load geojson
json_buildings = GeoJSONDataSource(geojson=json.dumps(geo_buildings))

In [43]:
def create_image(x_range=x_range, y_range=y_range, w=plot_w, h=plot_h):
    cvs = ds.Canvas(x_range=x_range, y_range=y_range, plot_height=h, plot_width=w)
    agg = cvs.line(df, 'lon', 'lat', agg=ds.count('c'))
    return tf.shade(agg, cmap=['#2C3539','#87ceff'])

In [44]:
#create_image()

In [45]:
def base_plot(tools='pan, wheel_zoom, reset, save'):
    p = bp.figure(tools=tools, plot_width=plot_w, plot_height=plot_h,
        x_range=x_range, y_range=y_range, outline_line_color=None,
        min_border=0, min_border_left=0, min_border_right=0,
        min_border_top=0, min_border_bottom=0)  
    p.patches(xs='xs', ys='ys', fill_color='#0C090A', fill_alpha=0.2, line_alpha=0, source=json_buildings)
    p.background_fill_color = '#2C3539'
    p.xaxis.visible = False
    p.yaxis.visible = False
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    return p

In [46]:
# Hard to figure out which colours look good ...
#bp.show(base_plot())

In [48]:
#p = base_plot()
#InteractiveImage(p, create_image)
# See "python_daily.png"